# AIM : Decision Tree ID3 Algorithm 

**Name : Trideep Nandi**

**Class : CS4**

**Batch: 2**

**Enrollment : 0827CS211248**

**Step by Step Decision Tree: ID3 Algorithm**
**Objectives:**
1. Knowing the basics of the ID3 Algorithm
2. Loading csv data in python, (using pandas library)
3. Training and building Decision tree using ID3 algorithm
4. Predicting from the tree
5. Finding out the accuracy

**Step 1: Observing The dataset**

https://www.kaggle.com/tareqjoy/trainplaytennis


**Step 2: Importing the necessary basic python libraries**

In [1]:
import pandas as pd #for manipulating the csv data
import numpy as np #for mathematical calculation

**Step 3: Reading the dataset**

In [4]:
train_data_m = pd.read_csv("content/PlayTennis.csv") #reading the dataset

train_data_m.head() #viewing some row of the dataset

,Outlook,Temperature,Humidity,Wind,Play Tennis
0,Sunny,Hot,High,Weak,No
1,Sunny,Hot,High,Strong,No
2,Overcast,Hot,High,Weak,Yes
3,Rain,Mild,High,Weak,Yes
4,Rain,Cool,Normal,Weak,Yes


**Step 4: Calculating the entropy of the whole dataset**

In [5]:
def calc_total_entropy(train_data, label, class_list):
    total_row = train_data.shape[0] #the total size of the dataset
    total_entr = 0

    for c in class_list: #for each class in the label
        total_class_count = train_data[train_data[label] == c].shape[0] #number of the class
        total_class_entr = - (total_class_count/total_row)*np.log2(total_class_count/total_row) #entropy of the class
        total_entr += total_class_entr #adding the class entropy to the total entropy of the dataset

    return total_entr

**Step 5: Calculating the entropy for the filtered dataset**

The purpose of this function is to calculate the total entropy of a dataset based on the given label and class list.

Parameters:
1. train_data: A pandas DataFrame representing the training dataset.
2. label: A string representing the name of the column that contains the label or target variable.
3. class_list: A list containing the unique class values present in the label column.

In [6]:
def calc_entropy(feature_value_data, label, class_list):
    class_count = feature_value_data.shape[0]
    entropy = 0

    for c in class_list:
        label_class_count = feature_value_data[feature_value_data[label] == c].shape[0] #row count of class c
        entropy_class = 0
        if label_class_count != 0:
            probability_class = label_class_count/class_count #probability of the class
            entropy_class = - probability_class * np.log2(probability_class)  #entropy
        entropy += entropy_class
    return entropy

**Step 6: Calculating information gain for a feature**

The purpose of this function is to calculate the entropy of a feature value (subset of data) based on the given label and class list.

Parameters:
1. feature_value_data: A pandas DataFrame representing a subset of the training data for a specific feature value.


In [7]:
def calc_info_gain(feature_name, train_data, label, class_list):
    feature_value_list = train_data[feature_name].unique() #unqiue values of the feature
    total_row = train_data.shape[0]
    feature_info = 0.0

    for feature_value in feature_value_list:
        feature_value_data = train_data[train_data[feature_name] == feature_value] #filtering rows with that feature_value
        feature_value_count = feature_value_data.shape[0]
        feature_value_entropy = calc_entropy(feature_value_data, label, class_list) #calculcating entropy for the feature value
        feature_value_probability = feature_value_count/total_row
        feature_info += feature_value_probability * feature_value_entropy #calculating information of the feature value

    return calc_total_entropy(train_data, label, class_list) - feature_info #calculating information gain by subtracting

**Step 8: Adding a node to the tree**

In [13]:
def generate_sub_tree(feature_name, train_data, label, class_list):
    feature_value_count_dict = train_data[feature_name].value_counts(sort=False) #dictionary of the count of unqiue feature value
    tree = {} #sub tree or node

    for feature_value, count in feature_value_count_dict.items():
        feature_value_data = train_data[train_data[feature_name] == feature_value] #dataset with only feature_name = feature_value

        assigned_to_node = False #flag for tracking feature_value is pure class or not
        for c in class_list: #for each class
            class_count = feature_value_data[feature_value_data[label] == c].shape[0] #count of class c

            if class_count == count: #count of (feature_value = count) of class (pure class)
                tree[feature_value] = c #adding node to the tree
                train_data = train_data[train_data[feature_name] != feature_value] #removing rows with feature_value
                assigned_to_node = True
        if not assigned_to_node: #not pure class
            tree[feature_value] = "?" #as feature_value is not a pure class, it should be expanded further,
                                      #so the branch is marking with ?

    return tree, train_data

**Step 7: Finding the most informative feature (feature with highest information gain)**

The purpose of this function is to generate a sub-tree (or node) for a specific feature in a decision tree.

Parameters:
1. feature_name: A string representing the name of the feature for which the sub-tree is being generated.
2. train_data: A pandas DataFrame representing the training dataset.
3. label: A string representing the name of the column that contains the label or target variable.
4. class_list: A list containing the unique class values present in the label column.
5. Feature Value Count Dictionary:
The function first creates a dictionary (feature_value_count_dict) that counts the occurrences of each unique feature value in the given feature column (feature_name).
6. Sub-Tree Generation:
For each unique feature value, the function creates a subset of the training data where the feature value matches the current unique value. Then it
Checks if the subset is a pure class (i.e., all instances in the subset belong to the same class).
Result:
The function returns a dictionary (tree) representing the sub-tree for the given feature. Each key in the dictionary corresponds to a unique feature value, and the value associated with that key can be either a class label or “?” (indicating further branching).

In [9]:
"""
def find_most_informative_feature(train_data, label, class_list):
 feature_value_list = ['Outlook','Wind', 'Temperature', 'Humidity']
 dict = {}
 for i in feature_value_list:
   dict[i]=calc_info_gain(i,train_data,label,class_list)
 v = list(dict.values())
 k = list(dict.keys())
 return(k[v.index(max(v))])
"""
def find_most_informative_feature(train_data, label, class_list):
  feature_list = train_data.columns.drop(label) #finding the feature names in the dataset
                                            #N.B. label is not a feature, so dropping it
  max_info_gain = -1
  max_info_feature = None

  for feature in feature_list:  #for each feature in the dataset
      feature_info_gain = calc_info_gain(feature, train_data, label, class_list)
      if max_info_gain < feature_info_gain: #selecting feature name with highest information gain
          max_info_gain = feature_info_gain
          max_info_feature = feature

  return max_info_feature

**Step 9: Performing ID3 Algorithm and generating Tree**

The function will now generate tree using ID3 decision tree algorithm.

In [10]:
def make_tree(root, prev_feature_value, train_data, label, class_list):
    if train_data.shape[0] != 0: #if dataset becomes empty after updating
        max_info_feature = find_most_informative_feature(train_data, label, class_list) #most informative feature
        tree, train_data = generate_sub_tree(max_info_feature, train_data, label, class_list) #getting tree node and updated dataset
        next_root = None

        if prev_feature_value != None: #add to intermediate node of the tree
            root[prev_feature_value] = dict()
            root[prev_feature_value][max_info_feature] = tree
            next_root = root[prev_feature_value][max_info_feature]
        else: #add to root of the tree
            root[max_info_feature] = tree
            next_root = root[max_info_feature]

        for node, branch in list(next_root.items()): #iterating the tree node
            if branch == "?": #if it is expandable
                feature_value_data = train_data[train_data[max_info_feature] == node] #using the updated dataset
                make_tree(next_root, node, feature_value_data, label, class_list) #recursive call with updated dataset

**Step 10: Finding unique classes of the label and Starting the algorithm**

In [11]:
def id3(train_data_m, label):
    train_data = train_data_m.copy() #getting a copy of the dataset
    tree = {} #tree which will be updated
    class_list = train_data[label].unique() #getting unqiue classes of the label
    make_tree(tree, None, train_data, label, class_list) #start calling recursion
    return tree

**Starting the algorithm**

In [14]:
tree = id3(train_data_m, 'Play Tennis')
print(tree)

{'Outlook': {'Sunny': {'Humidity': {'High': 'No', 'Normal': 'Yes'}}, 'Overcast': 'Yes', 'Rain': {'Wind': {'Weak': 'Yes', 'Strong': 'No'}}}}


**Step 11: Predicting from the tree**

In [15]:
def predict(tree, instance):
    if not isinstance(tree, dict): #if it is leaf node
        return tree #return the value
    else:
        root_node = next(iter(tree)) #getting first key/feature name of the dictionary
        feature_value = instance[root_node] #value of the feature
        if feature_value in tree[root_node]: #checking the feature value in current tree node
            return predict(tree[root_node][feature_value], instance) #goto next feature
        else:
            return None

**Step 12: Evaluating test dataset**

In [16]:
def evaluate(tree, test_data_m, label):
    correct_predict = 0
    wrong_predict = 0
    for index, row in test_data_m.iterrows(): #for each row in the dataset
        result = predict(tree, test_data_m.iloc[index]) #predict the row
        if result == test_data_m[label].iloc[index]: #predicted value and expected value is same or not
            correct_predict += 1 #increase correct count
        else:
            wrong_predict += 1 #increase incorrect count
    accuracy = correct_predict / (correct_predict + wrong_predict) #calculating accuracy
    return accuracy